# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [1]:
from sklearn import cross_validation, datasets, grid_search, linear_model, metrics

import numpy as np
import pandas as pd

C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Kate\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### Генерация датасета

In [2]:
iris = datasets.load_iris()

In [3]:
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

In [4]:
classifier = linear_model.SGDClassifier(random_state = 0)

### Генерация сетки

In [5]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'n_iter', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'verbose', 'warm_start'])

In [9]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'n_iter' : list(range(5,10)),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}

In [10]:
cv = cross_validation.StratifiedShuffleSplit(train_labels, n_iter = 10, test_size = 0.2, random_state = 0)

### Подбор параметров и оценка качества

#### Grid search

In [11]:
grid_cv = grid_search.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

In [12]:
%%time # time of function
grid_cv.fit(train_data, train_labels)

Wall time: 3.47 s


GridSearchCV(cv=StratifiedShuffleSplit(labels=[1 2 ..., 2 0], n_iter=10, test_size=0.2, random_state=0),
       error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'], 'penalty': ['l1', 'l2'], 'n_iter': [5, 6, 7, 8, 9], 'alpha': array([ 0.0001 ,  0.00032,  0.00055,  0.00078,  0.001  ])},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [13]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.00032499999999999999, average=False, class_weight=None,
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=9, n_jobs=1,
       penalty='l1', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False)

In [15]:
print (grid_cv.best_score_)
print (grid_cv.best_params_)

0.8952380952380953
{'alpha': 0.00032499999999999999, 'loss': 'hinge', 'n_iter': 9, 'penalty': 'l1'}


In [16]:
grid_cv.grid_scores_[:10]

[mean: 0.75714, std: 0.13544, params: {'alpha': 0.0001, 'loss': 'hinge', 'n_iter': 5, 'penalty': 'l1'},
 mean: 0.66667, std: 0.15936, params: {'alpha': 0.0001, 'loss': 'hinge', 'n_iter': 5, 'penalty': 'l2'},
 mean: 0.61429, std: 0.14357, params: {'alpha': 0.0001, 'loss': 'hinge', 'n_iter': 6, 'penalty': 'l1'},
 mean: 0.68571, std: 0.15386, params: {'alpha': 0.0001, 'loss': 'hinge', 'n_iter': 6, 'penalty': 'l2'},
 mean: 0.79524, std: 0.16503, params: {'alpha': 0.0001, 'loss': 'hinge', 'n_iter': 7, 'penalty': 'l1'},
 mean: 0.70952, std: 0.18982, params: {'alpha': 0.0001, 'loss': 'hinge', 'n_iter': 7, 'penalty': 'l2'},
 mean: 0.69524, std: 0.19772, params: {'alpha': 0.0001, 'loss': 'hinge', 'n_iter': 8, 'penalty': 'l1'},
 mean: 0.63810, std: 0.17587, params: {'alpha': 0.0001, 'loss': 'hinge', 'n_iter': 8, 'penalty': 'l2'},
 mean: 0.84286, std: 0.12608, params: {'alpha': 0.0001, 'loss': 'hinge', 'n_iter': 9, 'penalty': 'l1'},
 mean: 0.78095, std: 0.10690, params: {'alpha': 0.0001, 'loss': 

#### Randomized grid search

In [20]:
randomized_grid_cv = grid_search.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 20, 
                                                   random_state = 0) # estimate only for some combinations to find really bad ones

In [21]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

Wall time: 337 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(labels=[1 2 ..., 2 0], n_iter=10, test_size=0.2, random_state=0),
          error_score='raise',
          estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False),
          fit_params={}, iid=True, n_iter=20, n_jobs=1,
          param_distributions={'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'], 'penalty': ['l1', 'l2'], 'n_iter': [5, 6, 7, 8, 9], 'alpha': array([ 0.0001 ,  0.00032,  0.00055,  0.00078,  0.001  ])},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          scoring='accuracy', verbose=0)

In [22]:
print (randomized_grid_cv.best_score_)
print (randomized_grid_cv.best_params_)

0.8142857142857143
{'penalty': 'l1', 'n_iter': 9, 'loss': 'log', 'alpha': 0.00055000000000000003}
